In [ ]:
import numpy as np

np.random.seed(42)

n = 10_000

# 1) mayoría: distribución normal
expression = np.random.normal(loc=10, scale=2, size=n)

# 2) introducir ceros (por ejemplo 5%)
# la funcion choice espera un array, lista o entero. Si es entero lo transforma en array de 0 al valor pasado.
zero_idx = np.random.choice(n, size=int(0.05*n), replace=False)
expression[zero_idx] = 0

# 3) introducir outliers altos (por ejemplo 1%)
remaining_idx = np.setdiff1d(np.arange(n), zero_idx)
outliers_idx = np.random.choice(remaining_idx, size=int(0.01*n), replace=False)
expression[outliers_idx] += 30

#Otra forma de hacerlo usando where:
# mask_zero = np.random.rand(n) < 0.05
# expression = np.where(mask_zero, 0, expression)


In [13]:
mean_expression = np.mean(expression)
std_expression = np.std(expression)
p1_expression, p50_expression, p99_expression = np.percentile(expression, [1, 50, 99])

mean_expression, std_expression, p1_expression, p50_expression, p99_expression

(np.float64(9.795728165513445),
 np.float64(4.209778703312631),
 np.float64(0.0),
 np.float64(9.890926773143363),
 np.float64(17.87634560974353))

In [6]:
# Filtrado de outliers segun percentile 1 y 99
expression_filtered_mask = (expression >= p1_expression) & (expression <= p99_expression)
expression_filtered = expression[expression_filtered_mask]

In [ ]:
# Operación incorrecta a propósito: x/x (cuando x=0 -> 0/0 = NaN)
bad = expression / expression

# Esto detecta exactamente las posiciones donde expression era 0
zero_mask = np.isnan(bad)

# Comprobación rápida (opcional pero útil)
print("Ceros detectados:", zero_mask.sum())
print("Ceros reales:", (expression == 0).sum())

eps = 1e-8  # pequeño y > 0

log_expression = np.log(expression + eps)

print("Ejemplo (ceros ->):", log_expression[zero_mask][:5])
print("Ejemplo (otros valores ->):", log_expression[~zero_mask][:5])

In [16]:
def summarize(x):
    return {
        "mean": np.mean(x),
        "std": np.std(x),
        "p1": np.percentile(x, 1),
        "p50": np.percentile(x, 50),
        "p99": np.percentile(x, 99),
    }


In [17]:

summary_raw = summarize(expression)
summary_log = summarize(log_expression)

for key in summary_raw:
    print(
        f"{key:>4} | raw: {summary_raw[key]:>8.3f} | log: {summary_log[key]:>8.3f}"
    )

mean | raw:    9.796 | log:    1.260
 std | raw:    4.210 | log:    4.522
  p1 | raw:    0.000 | log:  -18.421
 p50 | raw:    9.891 | log:    2.292
 p99 | raw:   17.876 | log:    2.881


La transformación logarítmica reduce la asimetría y la influencia de valores extremos, 
lo que se refleja en una disminución de la dispersión y una mayor estabilidad de los percentiles.

La compresión logarítmica hace que las distancias reflejen cambios relativos en lugar de magnitudes absolutas, 
estabilizando la distribución y permitiendo comparaciones significativas.

English:
The logarithmic transformation reduces skewness and the influence of extreme values, which is reflected in a reduction of dispersion and greater stability of percentiles.

Logarithmic compression causes distances to reflect relative changes rather than absolute magnitudes, stabilizing the distribution and enabling meaningful comparisons.